<a href="https://colab.research.google.com/github/RonanKMcGovern/mpt/blob/main/Ronan_MPT_7B_Instruct_Long_Seq_Len.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install --upgrade transformers einops datasets accelerate

# Import

In [ ]:
import torch
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AdamW, AutoConfig
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from typing import Any, Dict, Tuple
import warnings
from threading import Event, Thread
import textwrap
from datasets import concatenate_datasets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Set up Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define cache directory
cache_dir = "/content/drive/MyDrive/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists

Mounted at /content/drive


# Mosaic Instruct Model MPT-7B-Instruct

## Original script (do not run)

In [ ]:
# Do not run - this is the old script.
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

class InstructionTextGenerationPipeline:
    def __init__(
        self,
        model_name,
        cache_dir,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:

        # use cache directory while loading model and tokenizer
        # Load the configuration
        config = transformers.AutoConfig.from_pretrained(model_name,
                                                        trust_remote_code=trust_remote_code)
        # Explicitly set the max_seq_len
        config.max_seq_len = 4096

        # Load the model with the updated configuration
        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            model_name,
            config=config,
            cache_dir=cache_dir,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            cache_dir=cache_dir,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right" # "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.5,
            "top_p": 0.92,
            "top_k": 0,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.1,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def format_instruction(self, instruction):
        return PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        s = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)
        input_ids = self.tokenizer(s, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # Slice the output_ids tensor to get only new tokens
        new_tokens = output_ids[0, len(input_ids[0]) :]
        output_text = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
        return output_text


## Custom Length MPT

In [ ]:
def initialize_parameters(
    max_seq_len=2048,
    max_output_len=1000,
    temperature=0.0
):
    max_input_len = max_seq_len - max_output_len #add a buffer
    return max_seq_len, max_output_len, max_input_len, temperature

max_seq_len, max_output_len, max_input_len, temperature = initialize_parameters()
print(f"Max sequence length: {max_seq_len}")
print(f"Max output length: {max_output_len}")
print(f"Max input length: {max_input_len}")
print(f"Temperature: {temperature}")

Max sequence length: 2048
Max output length: 1000
Max input length: 1048
Temperature: 0.0


In [ ]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

class InstructionTextGenerationPipeline:
    def __init__(
        self,
        model_name,
        cache_dir,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model_name = model_name
        self.cache_dir = cache_dir
        self.torch_dtype = torch_dtype
        self.trust_remote_code = trust_remote_code
        self.use_auth_token = use_auth_token

        # Load the configuration
        self.config = transformers.AutoConfig.from_pretrained(self.model_name, trust_remote_code=self.trust_remote_code)

        # Explicitly set the max_seq_len
        self.config.max_seq_len = max_seq_len
        # self.config.attn_config['attn_impl'] = 'triton'
        self.config.init_device = 'cuda:0' # For fast initialization directly on GPU!

    def load_model(self):
        # Load the model with the updated configuration
        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            self.model_name,
            config=self.config,
            cache_dir=self.cache_dir,
            torch_dtype=self.torch_dtype,
            trust_remote_code=self.trust_remote_code,
            use_auth_token=self.use_auth_token,
        )

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            self.model_name,
            cache_dir=self.cache_dir,
            trust_remote_code=self.trust_remote_code,
            use_auth_token=self.use_auth_token,
        )

        # Set the tokenizer's maximum length
        self.tokenizer.model_max_length = max_input_len #maybe should be set to the max_input_len

        if self.tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right" # "left"

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=self.torch_dtype)

        self.generate_kwargs = {
            "temperature": temperature,
            "top_p": 0.92,
            "top_k": 0,
            "max_new_tokens": max_output_len,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.1,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def format_instruction(self, instruction):
        return PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        s = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)
        input_ids = self.tokenizer(s, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # Slice the output_ids tensor to get only new tokens
        new_tokens = output_ids[0, len(input_ids[0]) :]
        output_text = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
        return output_text


In [ ]:
generate = InstructionTextGenerationPipeline(
    model_name="mosaicml/mpt-7b-instruct",
    cache_dir=cache_dir,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

generate.config  # check the config


MPTConfig {
  "_name_or_path": "mosaicml/mpt-7b-instruct",
  "architectures": [
    "MPTForCausalLM"
  ],
  "attn_config": {
    "alibi": true,
    "alibi_bias_max": 8,
    "attn_impl": "torch",
    "attn_pdrop": 0,
    "attn_type": "multihead_attention",
    "attn_uses_sequence_id": false,
    "clip_qkv": null,
    "prefix_lm": false,
    "qk_ln": false,
    "softmax_scale": null
  },
  "auto_map": {
    "AutoConfig": "mosaicml/mpt-7b-instruct--configuration_mpt.MPTConfig",
    "AutoModelForCausalLM": "mosaicml/mpt-7b-instruct--modeling_mpt.MPTForCausalLM"
  },
  "d_model": 4096,
  "emb_pdrop": 0,
  "embedding_fraction": 1.0,
  "expansion_ratio": 4,
  "init_config": {
    "emb_init_std": null,
    "emb_init_uniform_lim": null,
    "fan_mode": "fan_in",
    "init_div_is_residual": true,
    "init_gain": 0,
    "init_nonlinearity": "relu",
    "init_std": 0.02,
    "name": "kaiming_normal_",
    "verbose": 0
  },
  "init_device": "cuda:0",
  "learned_pos_emb": true,
  "logit_scale": nul

In [ ]:
generate.load_model()

Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/925e0d80e50e77aaddaf9c3ced41ca4ea23a1025/modeling_mpt.py
You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-30-77416838d325>:61: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


In [ ]:
print(generate.tokenizer)  # Print the tokenizer's configuration


GPTNeoXTokenizerFast(name_or_path='mosaicml/mpt-7b-instruct', vocab_size=50254, model_max_length=5144, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)


In [ ]:
  # Define stop_token_ids
stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

In [ ]:
  # Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def process_stream(instruction, temperature, top_p, top_k, max_new_tokens):

    def truncate_text(content, tokenizer, truncation_len):
      tokens = tokenizer.tokenize(content)
      print("Length of tokens before truncation: ", len(tokens))
      if len(tokens) > truncation_len:
          tokens = tokens[:truncation_len]
      print("Length of tokens after truncation: ", len(tokens))
      return tokenizer.convert_tokens_to_string(tokens)

    truncation_len = max_input_len - 500

    # Make sure your content fits into the model's max_length
    instruction = truncate_text(instruction, generate.tokenizer, truncation_len)

    ## The tokenizer isn't actually shortening the text properly. I need ot check truncate_text.

    # Tokenize the input
    input_ids = generate.tokenizer(
        generate.format_instruction(instruction), return_tensors="pt", truncation=True, max_length = max_input_len
    ).input_ids
    input_ids = input_ids.to(generate.model.device)

    print(generate.format_instruction(instruction)[:2500])

    print("Length of the tokenized input:", input_ids.shape[1])

    # Initialize the streamer and stopping criteria
    streamer = TextIteratorStreamer(
        generate.tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )
    stop = StopOnTokens()

    if temperature < 0.1:
        temperature = 0.0
        do_sample = False
    else:
        do_sample = True

    gkw = {
        **generate.generate_kwargs,
        **{
            "input_ids": input_ids,
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": do_sample,
            "top_p": top_p,
            "top_k": top_k,
            "streamer": streamer,
            "stopping_criteria": StoppingCriteriaList([stop]),
        },
    }

    response = ''

    def generate_and_signal_complete():
        generate.model.generate(**gkw)

    t1 = Thread(target=generate_and_signal_complete)
    t1.start()

    for new_text in streamer:
        response += new_text

    return response

In [ ]:
def generate_text(instruction, temperature = temperature, top_p = 0.95, top_k = 0, max_new_tokens = max_output_len):
  response = process_stream(instruction, temperature, top_p, top_k, max_new_tokens)

  wrapped_text = textwrap.fill(response, width=100)
  print(wrapped_text)

In [ ]:
generate_text('Summarise the following text: \n\nwe are here live in Omaha Nebraska good morning everybody I m Becky quick along with Mike santoli and in just 30 minutes time Berkshire Hathaway chairman and CEO Warren Buffett s going to be taking the stage with his vice chair Charlie Munger the legendary duo will also be joined by berkshire s two other')

Length of tokens before truncation:  76
Length of tokens after truncation:  76
337
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Summarise the following text: 

we are here live in Omaha Nebraska good morning everybody I m Becky quick along with Mike santoli and in just 30 minutes time Berkshire Hathaway chairman and CEO Warren Buffett s going to be taking the stage with his vice chair Charlie Munger the legendary duo will also be joined by berkshire s two other
### Response:

Length of the tokenized input: 104
Warren buffet, one of the most successful businessmen ever known for investing in companies like
Apple or Coca Cola among many others has been invited as keynote speaker at this year's annual
meeting of shareholders where he'll share insights on how businesses can thrive during these
uncertain times


In [ ]:
import re


# Read the file content
with open(os.path.join(cache_dir, 'berkshire23.txt'), 'r') as f:
    content = f.read()

generate_text(f'Provide a detailled summary of the following: \n\n{content}') # this is truncating off the very end of the response.


Length of tokens before truncation:  78790
Length of tokens after truncation:  548
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Provide a detailled summary of the following: 

we are here live in Omaha Nebraska good morning everybody I'm Becky quick
along with Mike santoli and in just 30 minutes time Berkshire Hathaway chairman and CEO Warren Buffett's going to be
taking the stage with his vice chair Charlie Munger the legendary duo will also be joined by berkshire's two other
Vice chairs Greg Abel who manages the non-insurance operations for the company and Ajit Jain who runs all of the
insurance businesses and as always it's pretty big crowd here lots and lots of people and a few people you might notice
too Tim Cook is here Apple of course is still berkshire's largest holding big big part of its portfolio there you see
him backstage getting ready to go out and take his seat he gets to sit down in the specia

In [ ]:
print(max_input_len)

import re

# Define the text cleanup function
def clean_text(text):
    return re.sub(r"[^\w\s]", " ", text)

# Define the truncate_text function // this probably isn't needed because the tokenizer does it...

truncation_len = max_input_len - 500

print(truncation_len)

def truncate_text(content, tokenizer, truncation_len):
    tokens = tokenizer.tokenize(content)
    print("Length of tokens before truncation: ", len(tokens))
    if len(tokens) > truncation_len:
        tokens = tokens[:truncation_len]
    print("Length of tokens after truncation: ", len(tokens))
    return tokenizer.convert_tokens_to_string(tokens)

# Read the file content
with open(os.path.join(cache_dir, 'berkshire23.txt'), 'r') as f:
    content = f.read()

# Clean up special characters in the content
content = clean_text(content)

# Define your instruction
instruction_end = "\n\n Summarise the above text."

# Make sure your content fits into the model's max_length
content = truncate_text(content, generate.tokenizer, truncation_len)

# Now form your full instruction
instruction = f"{content} {instruction_end}"

import textwrap

print("\n".join(textwrap.wrap(instruction, width=80)))


3096
2596
Length of tokens before truncation:  78582
Length of tokens after truncation:  2596
we are here live in Omaha Nebraska good morning everybody I m Becky quick along
with Mike santoli and in just 30 minutes time Berkshire Hathaway chairman and
CEO Warren Buffett s going to be taking the stage with his vice chair Charlie
Munger the legendary duo will also be joined by berkshire s two other Vice
chairs Greg Abel who manages the non insurance operations for the company and
Ajit Jain who runs all of the insurance businesses and as always it s pretty big
crowd here lots and lots of people and a few people you might notice too Tim
Cook is here Apple of course is still berkshire s largest holding big big part
of its portfolio there you see him backstage getting ready to go out and take
his seat he gets to sit down in the special seats by the way that s Debbie
pasonic Warren s assistant who s standing by just went bite beside him also in
the crowd Bill Murray he has been here for a cou

In [ ]:
generate_text(instruction)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
we are here live in Omaha Nebraska good morning everybody I m Becky quick
along with Mike santoli and in just 30 minutes time Berkshire Hathaway chairman and CEO Warren Buffett s going to be
taking the stage with his vice chair Charlie Munger the legendary duo will also be joined by berkshire s two other
Vice chairs Greg Abel who manages the non insurance operations for the company and Ajit Jain who runs all of the
insurance businesses and as always it s pretty big crowd here lots and lots of people and a few people you might notice
too Tim Cook is here Apple of course is still berkshire s largest holding big big part of its portfolio there you see
him backstage getting ready to go out and take his seat he gets to sit down in the special seats by the way that s
Debbie pasonic Warren s assistant who s standing by just went bite beside him also in the crowd Bill Murr

# Data Processing

In [ ]:
dataset = load_dataset('ag_news')
# Extract label names
label_names = dataset['train'].features['label'].names
label_names

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

['World', 'Sports', 'Business', 'Sci/Tech']

In [ ]:
label_names_str = str(label_names).replace("'","")
prefix = f"Classify the following text into one of the following categories: {label_names_str}\nText: "

num_samples_per_class = 75  # Set this to determine the number of samples per class

# Extract subsets of the training dataset with equal number of samples per class
train_datasets = []
for label in range(len(label_names)):
    label_dataset = dataset['train'].filter(lambda x: x['label'] == label).select(range(num_samples_per_class))
    label_dataset = label_dataset.map(lambda x: {'text': generate.format_instruction(prefix + x['text']) + label_names[x['label']]})
    train_datasets.append(label_dataset)

train_dataset = concatenate_datasets(train_datasets)

# Do the same for the validation dataset, if needed
val_datasets = []
for label in range(len(label_names)):
    label_dataset = dataset['test'].filter(lambda x: x['label'] == label).select(range(num_samples_per_class))
    label_dataset = label_dataset.map(lambda x: {'text': generate.format_instruction(prefix + x['text']) + label_names[x['label']]})
    val_datasets.append(label_dataset)

val_dataset = concatenate_datasets(val_datasets)

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

In [ ]:
# Tokenize the datasets
train_encodings = generate.tokenizer(train_dataset['text'], truncation=True, padding=True, max_length=256)
val_encodings = generate.tokenizer(val_dataset['text'], truncation=True, padding=True, max_length=256)

In [ ]:
# Convert the encodings to PyTorch datasets
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)
val_dataset = TextDataset(val_encodings)

Example Before Fine Tuning

In [ ]:
index = 0
example_text = prefix + dataset['test'][index]['text']
print('Example text:')
print(textwrap.fill(example_text, width=100))
print("Response:")
generate_text(example_text, temperature=0)
print("Correct Label:")
print(label_names[dataset['test'][index]['label']])

Example text:
Classify the following text into one of the following categories: [World, Sports, Business,
Sci/Tech] Text: Fears for T N pension after talks Unions representing workers at Turner   Newall say
they are 'disappointed' after talks with stricken parent firm Federal Mogul.
Response:
The above paragraph can be classified as follows - World (Fears), Sport(Unions),Business and Science
& Tech
Correct Label:
Business


# Freeze Layers

In [ ]:
for name, param in generate.model.named_parameters():
    print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
    if "31" not in name:
        param.requires_grad = False
    print(name, param.requires_grad)

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


In [ ]:
# Create the Trainer and train
trainer = Trainer(
    model=generate.model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,3.017200
20,3.056200
30,2.979700
40,2.846900
50,2.607800
60,2.360900
70,2.101600
80,1.918700
90,1.705500
100,1.498400


TrainOutput(global_step=950, training_loss=0.9318174342105263, metrics={'train_runtime': 239.8278, 'train_samples_per_second': 62.545, 'train_steps_per_second': 3.961, 'total_flos': 9.509450563584e+16, 'train_loss': 0.9318174342105263, 'epoch': 50.0})

In [ ]:
generate.model.eval()

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): Embedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

Example After Fine Tuning

In [ ]:
index = 0
example_text = prefix + dataset['test'][index]['text']
print('Example text:')
print(textwrap.fill(example_text, width=100))
print("Response:")
generate_text(example_text, temperature=0)
print("Correct Label:")
print(label_names[dataset['test'][index]['label']])

Example text:
Classify the following text into one of the following categories: [World, Sports, Business,
Sci/Tech] Text: Fears for T N pension after talks Unions representing workers at Turner   Newall say
they are 'disappointed' after talks with stricken parent firm Federal Mogul.
Response:
Business
Correct Label:
Business
